In [ ]:
import pandas as pd
import matplotlib as plt
import seaborn as sns

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline

In [ ]:
data = pd.read_csv('../input/predict-test-scores-of-students/test_scores.csv')

x = data.drop(columns=['posttest', 'student_id'])
y = data['pretest']

data.head()

In [ ]:
x = data.drop(columns=['posttest', 'student_id'])

schools = x.school.unique()
print("There are", len(schools), "schools, which are \n", schools, "\n")

areas = x.school_setting.unique()
print("There are ", len(areas), "areas, which are \n", areas, "\n")

classrooms = x.classroom.unique()
print("There are", len(classrooms), "kinds of classooms, which are \n", classrooms, "\n")

x.head()

In [ ]:
x = data.drop(columns=['posttest', 'student_id'])

#preprocessing

#helper class for create dictionary
class my_dict(dict):
    def __init__(self):
        self = dict()
        
    def add(self, key, value):
        self[key] = value
        
school_mapping = my_dict()
for i in range(len(schools)):
    school_mapping.add(schools[i], i)

classroom_mapping = my_dict()
for j in range(len(classrooms)):
    classroom_mapping.add(classrooms[j], j)
    
def student_mapping(num):
    if num<20:
        return 0
    elif num<25:
        return 1
    elif num<32:
        return 2
    
x['gender'] = x['gender'].apply(lambda x: 0 if x=='Male' else 1)
x['school_setting'] = x['school_setting'].map({'Urban':0, 'Suburban':1, 'Rural':2})
x['school'] = x['school'].map(school_mapping)
x['school_type'] = x['school_type'].apply(lambda x: 0 if x=='Public' else 1)
x['teaching_method'] = x['teaching_method'].apply(lambda x: 0 if x=="Standard" else 1)
x['lunch'] = x['lunch'].apply(lambda x: 0 if x=="Does not qualify" else 1)
x['classroom'] = x['classroom'].map(classroom_mapping)
x['n_student'] = x['n_student'].apply(student_mapping)

x.head()

In [ ]:
corr = x
corr['posttest'] = data['posttest']
corr = corr.corr() 

ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(x, data['posttest'], random_state=1)

xgb_model = xgb.XGBClassifier(
    #learning_rate = 0.02,
    n_estimators= 2000,
    max_depth= 4,
    min_child_weight= 2,
    #gamma=1,
    gamma=0.9,                        
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread= -1,
    scale_pos_weight=1)

xgb_model.fit(train_x, train_y) 

In [ ]:
pred = xgb_model.predict(test_x)

acc = accuracy_score(pred, test_y)
print("the model has", acc, "accuracy")